In [1]:
#Load the ROOT libraries
import ROOT

ERROR in cling::CIFactory::createCI(): cannot extract standard library include paths!
Invoking:
  LC_ALL=C x86_64-conda-linux-gnu-c++  -O3 -DNDEBUG -xc++ -E -v /dev/null 2>&1 | sed -n -e '/^.include/,${' -e '/^ \/.*++/p' -e '}'
Results was:
With exit code 0
C system headers (glibc/Xcode/Windows SDK) must be installed.
In file included from input_line_4:36:
/../lib/gcc/x86_64-linux-gnu/11/../../../../include/c++/11/cassert:44:10: fatal error: 'assert.h' file not found
#include <assert.h>
         ^~~~~~~~~~
input_line_37:1:10: fatal error: 'dlfcn.h' file not found
#include "dlfcn.h"
         ^~~~~~~~~


Most syntax is very similar, i.e. opening a TFile and inspecting it

In [2]:
f = ROOT.TFile('~/ROOT/example_tree.root', 'open')
f.ls()

TFile**		/home/jovyan/ROOT/example_tree.root	
 TFile*		/home/jovyan/ROOT/example_tree.root	
  KEY: TTree	tree;1	


When we extract the tree, we just have to make a variable to hold it.
The python layer handles the type conversions etc.

In [3]:
tree = f.Get('tree')
tree.Show()

======> EVENT:-1
 event_number    = 0
 momentum        = 0
 nproducts       = 0
 interacted      = 0
 interaction_type = NULL
 r1              = 0
 r2              = 0
 r3              = 0
 product_rs      = NULL


TCanvas is a class we haven't seen yet -- however, it is important for visualization.
It is in fact used within TBrowser. We need to use it here because we can't use TBrowser
in this jupyter notebook

After we make it, we can draw to it from the tree (and later a histogram). Then we need to draw the canvas it self

In [31]:
c = ROOT.TCanvas()
tree.Draw("momentum")
c.Draw()

Let's determine the interaction rate as a function of momentum

In [33]:
c = ROOT.TCanvas()
tree.Draw('momentum>>hAll(50, 4.5, 5.5)')
tree.Draw('momentum>>hInteracted(50, 4.5, 5.5)', 'interacted')
hAll = ROOT.gDirectory.Get('hAll')
hInteracted = ROOT.gDirectory.Get('hInteracted')
hInteracted.Divide(hAll)
hInteracted.Draw()
c.Draw()

So far, things look very similar to C++, other than not having to specify types & not having to deal with pointers (everything in python is basically a pointer)

Things start to get more interesting when looking at TTrees.

In [6]:
for entry in tree: # We can iterate over the entries in the TTree
    if (entry.event_number > 10): break # We can access a TBranch as if it were a member of the entry
    print(entry.event_number)

0
1
2
3
4
5
6
7
8
9
10


In [7]:
for entry in tree:
    if (entry.event_number > 10): break

    #Working with container (i.e. vector) TBranches is pretty easy:
    print(len(entry.product_rs), entry.product_rs)
    if len(entry.product_rs) > 0: print(entry.product_rs[0])

    # And we can work with these containers with list comprehension
    total = sum([i for i in entry.product_rs if i > 0.])
    print('Total', total)
    print()

0 {}
Total 0

0 {}
Total 0

0 {}
Total 0

12 { -0.90962027, -0.23765899, -0.73710196, 0.70613013, 1.5319749, 1.3373288, 0.62758713, 0.32251756, -2.9322923, -0.73837203, 1.0242677, -0.79618540 }
-0.9096202712624284
Total 5.549806178280907

0 {}
Total 0

4 { -0.46482704, -0.012503151, -2.1786854, -0.55582049 }
-0.4648270420730114
Total 0

5 { 1.9453188, 0.52476631, -1.6176688, 1.2510964, -0.25610404 }
1.9453187986229998
Total 3.721181514761363

3 { -1.2426485, 0.42245057, 0.45433255 }
-1.2426484855823219
Total 0.8767831222483342

7 { -1.0686483, -0.72064190, 1.0341846, 0.57350768, 0.74360996, 0.063696116, 1.1339623 }
-1.0686482824336805
Total 3.5489606531336904

1 { 0.69258886 }
0.6925888634286821
Total 0.6925888634286821

5 { -1.0900181, 2.4187389, -0.23679585, 0.050562125, -1.4414020 }
-1.0900181084871292
Total 2.469300998203232



There is some additional complications if we want to build a TTree

In [8]:
from array import array

In [23]:
#Need to use array.array or numpy.array to make branches
from array import array
r = array('i', [0])
r5 = array('i', 5*[0]) #This makes a length-5 integer array
fout = ROOT.TFile('test_python.root', 'recreate')
tout = ROOT.TTree('out_tree', '')
tout.Branch('r', r, 'r/I') #Note: need to specify the leaf (3rd arg)
tout.Branch('r5', r5, 'r5[5]/I') #Note: need to specify the length of the array
for i in range(10):
    r[0] = i
    for j in range(5): r5[j] = i*j
    tout.Fill()
tout.Write()

440

In [24]:
tout.Show()

======> EVENT:-1
 r               = 9
 r5              = 0, 
                  9, 18, 27, 36


In [35]:
c = ROOT.TCanvas()
tout.Draw('r')
c.Draw()

In [36]:
c = ROOT.TCanvas()
tout.Draw('r5:r', '', 'colz')
c.Draw()